In [3]:
import networkx as nx
import pandas as pd
import numpy as np
from math import sin,cos,sqrt,atan2
import pickle
import matplotlib.pyplot as plt
%matplotlib inline  
#import utm
import geopy.distance
from datetime import datetime

# Network data and demand data

In [17]:
G = pickle.load( open( "Manhattan_network.p", "rb" ) )

In [2]:
dic = dict(nx.all_pairs_dijkstra_path_length(G,weight = 'length'))


In [4]:
#pickle.dump( dic, open( "dic.p", "wb" ))
#pickle.dump( demand_station, open( "demand_station_30s.p", "wb" ))

In [6]:
#data = pd.read_csv("/Users/xiaotong/Desktop/CEE 6620/yellow_tripdata_2015-01.csv")
data = pd.read_csv("/Users/barryfan/Downloads/yellow_tripdata_2015-01.csv")

In [8]:
data_1 = data[['tpep_pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']]

In [10]:
sorted_data = data_1.sort_values(['tpep_pickup_datetime'], ascending=[True])

In [284]:
drop_list = []
for i,demand in enumerate(sorted_data.values):
    if demand[0] < '2015-01-15 08:00:00' or demand[0] >= '2015-01-15 08:00:30':
        drop_list.append(i)
demand = np.delete(sorted_data.values, drop_list, axis = 0)

# Assign demand to road stations

In [18]:
station_id = {}
for node in G.nodes:
    longitude = G.nodes[node]['x']
    latitude = G.nodes[node]['y']
    node_id = G.node[node]['osmid']
    station_id[node_id] = [longitude,latitude]

In [286]:
demand_station = []
demand_time = dict()
for d in demand:
    ori = (d[1],d[2])
    dest = (d[3],d[4])
    d_ori = []
    d_dest = []
    for node in G.nodes:
        node_id = G.node[node]['osmid']
        node2 = (G.nodes[node]['x'],G.nodes[node]['y'])
        ori_distance = geopy.distance.vincenty(ori, node2).km
        dest_distance = geopy.distance.vincenty(dest, node2).km
        d_ori.append((ori_distance,node_id))
        d_dest.append((dest_distance,node_id))
    d_ori.sort()
    d_dest.sort()
    if d_ori[0][0] >= 0.3 or d_dest[0][0] >= 0.3:
        continue
    ori_station = d_ori[0][1]
    dest_station = d_dest[0][1]
    demand_station.append([ori_station,dest_station])
    demand_time[(ori_station,dest_station)] = d[0]

In [287]:
len(demand_station)

182

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline  
import utm
def visualize_demand(demand,G,dic):
    pos = {}
    for node in G.nodes():
        longitude = G.nodes[node]['x']
        latitude = G.nodes[node]['y']
        x, y, a, b = utm.from_latlon(longitude,latitude)
        pos[node] = (x, y)
    fig = plt.figure(figsize=(20, 10))
    nx.draw_networkx(G, pos, fig_height=10, fig_width=20, node_size=0, with_labels=False, linewidths=0.1, alpha=0.1)
    for d in demand:
        ori = d[0]
        dest = d[1]
        x1, y1, a1, b1 = utm.from_latlon(dic[ori][0],dic[ori][1])
        x2, y2, a2, b2 = utm.from_latlon(dic[dest][0],dic[dest][1])
        plt.scatter(x1, y1, s=30 , c='r',marker = 'o')
        plt.scatter(x2, y2, s=30 , c='g',marker = 'o')
        plt.plot([x1,x2], [y1,y2], 'bo--', alpha=0.7, linewidth=1,markersize = 1)

In [ ]:
## For visuliazation - DO NOT RUN
visualize_demand(demand_station,G,station_id)

# Find meeting points

In [38]:
max_walking_distance = 300 #m
max_delay = 300 #s
max_waiting = 300 #s

Waking function $F = 150 \cdot \gamma$, $\gamma \in [0,2]$.

In [288]:
# Find meeting point with corresponding incentive pricing for each demand
pick_meeting = {}
drop_meeting = {}
demand_dic = {}
for i,demand in enumerate(demand_station):
    demand_dic[i] = demand
    demand_pick_location = demand[0]
    demand_drop_location = demand[1]
    pick_meeting[i] = []
    drop_meeting[i] = []
    for node in G.nodes:
        node_id = G.node[node]['osmid']
        ori_distance = dic[demand_pick_location][node_id]
        dest_distance = dic[demand_drop_location][node_id]
        if ori_distance <= max_walking_distance:
            incentive = ori_distance/150
            pick_meeting[i].append([node_id,incentive])
        if dest_distance <= max_walking_distance:
            incentive = dest_distance/150
            drop_meeting[i].append([node_id,incentive])

In [232]:
## For Comparision - DO NOT RUN
pick_meeting = {}
drop_meeting = {}
demand_dic = {}
for i,demand in enumerate(demand_station):
    demand_dic[i] = demand
    demand_pick_location = demand[0]
    demand_drop_location = demand[1]
    pick_meeting[i] = []
    drop_meeting[i] = []
    for node in G.nodes:
        node_id = G.node[node]['osmid']
        ori_distance = dic[demand_pick_location][node_id]
        dest_distance = dic[demand_drop_location][node_id]
        if ori_distance <= max_walking_distance:
            incentive = ori_distance/150
            if incentive != 0:
                continue
            pick_meeting[i].append([node_id,incentive])
        if dest_distance <= max_walking_distance:
            incentive = dest_distance/150
            if incentive != 0:
                continue
            drop_meeting[i].append([node_id,incentive])

In [25]:
def time_dif(demand_time, demand1, demand2):
    '''
    return time(demand2) - time(demand1) in seconds
    '''
    time1 = demand_time[(demand1[0], demand1[1])]
    time2 = demand_time[(demand2[0], demand2[1])]
    datetime_object1 = datetime.strptime(time1[-8:],'%H:%M:%S')
    datetime_object2 = datetime.strptime(time2[-8:],'%H:%M:%S')
    return (datetime_object2 - datetime_object1).total_seconds()

In [173]:
def check_feasibility(demand_a,demand_b,max_delay,max_waiting,dic,demand_time,
                      pk_stop=60, dp_stop=60):
    """
    Output: T/F, total savings, total incentives, which case
    """
    vw = 5*1000/3600 #walking speed in m/s
    vd = 20*1000/3600 #driving speed in m/s
    # dictionary: [pk/dp point: [incentive_a, incentive_b]]
    comm_pk = dict()
    comm_dp = dict()
    # because i used wrong variable name so it's grandfathered in and they are both ok...
    pk_time = pk_stop
    dp_time = dp_stop
    for i in pick_meeting[demand_a]:
        for j in pick_meeting[demand_b]:
            if i[0] == j[0]:
                comm_pk[i[0]] = [i[1], j[1]]
    for i in drop_meeting[demand_a]:
        for j in drop_meeting[demand_b]:
            if i[0] == j[0]:
                comm_dp[i[0]] = [i[1], j[1]]
    num_pk = len(comm_pk)
    num_dp = len(comm_dp)
    station_a = demand_station[demand_a]
    station_b = demand_station[demand_b]
    # vehicle time - does not include stop time
    time_a_vh = dic[station_a[0]][station_a[1]]/vd
    time_b_vh = dic[station_b[0]][station_b[1]]/vd
    # a passenger only experiences delay in pick_up but not drop_off. The system will however experience both.
    time_a_ns = time_a_vh + pk_stop #time if not sharing
    time_b_ns = time_b_vh + pk_stop
    time_diff = abs(time_dif(demand_time, station_a, station_b)) #difference in request time in seconds
    if num_pk != 0 and num_dp != 0:
        # time wasted in stopping: for passenger only pick_up time is wasted
        ij_walking = dict() # total walking time if they choose pickup i and dropoff j
        for i in comm_pk:
            waiting = time_diff + abs(dic[station_b[0]][i]/vw - dic[station_a[0]][i]/vw)
            for j in comm_dp:
                time_ij = dic[i][j]/vd
                delay = np.zeros(2)
                delay[0] = dic[station_a[0]][i]/vw + time_ij + dic[j][station_a[1]]/vw + pk_stop - time_a_ns
                delay[1] = dic[station_b[0]][i]/vw + time_ij + dic[j][station_b[1]]/vw + pk_stop - time_b_ns
                if max(delay) <= max_delay and waiting <= max_waiting:
                    ij_walking[(i, j)] = (dic[station_a[0]][i] + dic[j][station_a[1]] + dic[station_b[0]][i]
                                          + dic[j][station_b[1]])/vw
        if len(ij_walking) != 0:
            # find the best pickup and dropoff meeting points
            pk = 0
            dp = 0
            t_w = 9999
            for pair in ij_walking:
                if ij_walking[pair] < t_w:
                    t_w = ij_walking[pair]
                    pk = pair[0]
                    dp = pair[1]
            savings = time_a_vh + time_b_vh - dic[pk][dp]/vd + pk_stop + dp_stop # saved a pick_up and a drop_off
            # find incentives
            incentives = comm_pk[pk][0] + comm_pk[pk][1] + comm_dp[dp][0] + comm_dp[dp][1]
            return True, savings, incentives, 0
    if num_pk != 0:
        #i_walking will have two elements: total time walking and drop off a first(0) or b first(1)
        i_walking = dict()
        for i in comm_pk:
            walk_a = dic[station_a[0]][i]/vw
            walk_b = dic[station_b[0]][i]/vw
            if time_diff + abs(walk_b - walk_a) <= max_waiting:
                delay_1 = np.zeros(2)
                delay_2 = np.zeros(2)
                # 1st case: go to a's destination first
                delay_1[0] = walk_a + dic[i][station_a[1]]/vd + pk_stop - time_a_ns
                delay_1[1] = walk_b + (dic[i][station_a[1]] + 
                                       dic[station_a[1]][station_b[1]])/vd + pk_stop + dp_stop - time_b_ns
                # 2nd case: go to b's destination first
                delay_2[0] = walk_a + (dic[i][station_b[1]] + 
                                       dic[station_b[1]][station_a[1]])/vd + pk_stop + dp_stop - time_a_ns
                delay_2[1] = walk_b + dic[i][station_b[1]]/vd + pk_stop - time_b_ns
                # if either case is fine, choose the case with minimum walking
                if max(delay_1) <= max_delay and max(delay_2) <= max_delay:
                    if walk_a <= walk_b:
                        i_walking[i] = [walk_a + walk_b, 0]
                    else:
                        i_walking[i] = [walk_a + walk_b, 1]
                elif max(delay_1) <= max_delay:
                    i_walking[i] = [walk_a + walk_b, 0]
                elif max(delay_2) <= max_delay:
                    i_walking[i] = [walk_a + walk_b, 1]
        if len(i_walking) != 0:
            pk = 0
            t_w = 9999
            for i in i_walking:
                if i_walking[i][0] < t_w:
                    t_w = i_walking[i][0]
                    pk = i
            if i_walking[pk][1] == 0: # if dropoff a first
                savings = (time_a_ns + time_b_ns - (dic[pk][station_a[1]] 
                                                    + dic[station_a[1]][station_b[1]])/vd - pk_stop)
                # only saved a pick up stop time
            else:
                savings = time_a_vh + time_b_vh - (dic[pk][station_b[1]] + 
                                                   dic[station_b[1]][station_a[1]])/vd + pk_stop
            incentives = comm_pk[pk][0] + comm_pk[pk][1]
            return True, savings, incentives, 1
    if num_dp != 0: # last case: common dropoff point
        #j_walking will have two elements: total time walking and pick up a first(0) or b first(1)
        j_walking = dict()
        for j in comm_dp:
            walk_a = dic[j][station_a[1]]/vw
            walk_b = dic[j][station_b[1]]/vw
            delay_1 = np.zeros(2)
            delay_2 = np.zeros(2)
            # 1st case: pick up a first
            delay_1[0] = (dic[station_a[0]][station_b[0]] + 
                          dic[station_b[0]][j])/vd + walk_a + 2*pk_stop - time_a_ns
            delay_1[1] = dic[station_b[0]][j]/vd + walk_b + pk_stop - time_b_ns
            if time_diff + pk_stop + dic[station_a[0]][station_b[0]]/vd > max_waiting:
                # if watiting exceeds max_waiting, disqualify this match by making delay = 9999
                delay_1[1] = 9999
            # 2nd case: pick up b first
            delay_2[0] = dic[station_a[0]][j]/vd + walk_a + pk_stop - time_a_ns
            delay_2[1] = (dic[station_b[0]][station_a[0]] + 
                          dic[station_a[0]][j])/vd + walk_b + 2*pk_stop - time_b_ns
            if time_diff + dic[station_b[0]][station_a[0]]/vd > max_waiting:
                delay_2[0] = 9999
            # if either case is fine, choose the case with minimum walking
            if max(delay_1) <= max_delay and max(delay_2) <= max_delay:
                if walk_a <= walk_b:
                    j_walking[j] = [walk_a + walk_b, 0]
                else:
                    j_walking[j] = [walk_a + walk_b, 1]
            elif max(delay_1) <= max_delay:
                j_walking[j] = [walk_a + walk_b, 0]
            elif max(delay_2) <= max_delay:
                j_walking[j] = [walk_a + walk_b, 1]
        if len(j_walking) != 0:
            dp = 0
            t_w = 9999
            for j in j_walking:
                if j_walking[j][0] < t_w:
                    t_w = j_walking[j][0]
                    dp = j
            # only saved a drop_off time
            if j_walking[dp][1] == 0:
                savings = time_a_vh + time_b_vh - (dic[station_a[0]][station_b[0]] + 
                                                   dic[station_b[0]][j])/vd + dp_stop
            else:
                savings = time_a_vh + time_b_vh - (dic[station_b[0]][station_a[0]] + 
                                                   dic[station_a[0]][j])/vd + dp_stop
            incentives = comm_dp[dp][0] + comm_dp[dp][1]
            return True, savings, incentives, 2
    #consider no-meeting-point share if non of the meeting point cases work
    delays = np.zeros(4)
    savings = dict()
    # 1-2-1-2. b experiences a drop_off delay when a got off
    delay_1 = np.zeros(2)
    delay_1[0] = 2*pk_time + (dic[station_a[0]][station_b[0]] + 
                              dic[station_b[0]][station_a[1]])/vd - time_a_ns
    delay_1[1] = pk_time + (dic[station_b[0]][station_a[1]] + 
                            dic[station_a[1]][station_b[1]])/vd + dp_time - time_b_ns
    waiting = time_diff + pk_stop + dic[station_a[0]][station_b[0]]/vd
    delays[0] = max(delay_1)
    if waiting > max_waiting:
        delays[0] = 9999
    savings[delays[0]] = time_a_vh + time_b_vh - (dic[station_a[0]][station_b[0]] + 
                                                  dic[station_b[0]][station_a[1]] + 
                                                  dic[station_a[1]][station_b[1]])/vd
    # 1-2-2-1
    delay_2 = np.zeros(2)
    delay_2[0] = 2*pk_time + (dic[station_a[0]][station_b[0]] + dic[station_b[0]][station_b[1]] + 
                              dic[station_b[1]][station_a[1]])/vd + dp_time - time_a_ns
    
    delay_2[1] = pk_time + dic[station_b[0]][station_b[1]]/vd - time_b_ns
    waiting = time_diff + pk_stop + dic[station_a[0]][station_b[0]]/vd
    delays[1] = max(delay_2)
    if waiting > max_waiting:
        delays[1] = 9999
    savings[delays[1]] = time_a_vh + time_b_vh - (dic[station_a[0]][station_b[0]] + 
                                                  dic[station_b[0]][station_b[1]] + 
                                                  dic[station_b[1]][station_a[1]])/vd
     # 2-1-2-1
    delay_3 = np.zeros(2)
    delay_3[0] = pk_time + (dic[station_a[0]][station_b[1]] + 
                            dic[station_b[1]][station_a[1]])/vd + dp_time - time_a_ns
    delay_3[1] = 2*pk_time + (dic[station_b[0]][station_a[0]] + 
                              dic[station_a[0]][station_b[1]])/vd - time_b_ns
    waiting = time_diff + pk_stop + dic[station_b[0]][station_a[0]]/vd
    delays[2] = max(delay_3)
    if waiting > max_waiting:
        delays[2] = 9999
    savings[delays[2]] = time_a_vh + time_b_vh - (dic[station_b[0]][station_a[0]] + 
                                                  dic[station_a[0]][station_b[1]] + 
                                                  dic[station_b[1]][station_a[1]])/vd
    # 2-1-1-2
    delay_4 = np.zeros(2)
    delay_4[0] = pk_time + dic[station_a[0]][station_b[0]]/vd - time_a_ns
    delay_4[1] = 2*pk_time + (dic[station_b[0]][station_a[0]] + dic[station_a[0]][station_a[1]] + 
                              dic[station_a[1]][station_b[1]])/vd + dp_time - time_b_ns
    waiting = time_diff + pk_stop + dic[station_b[0]][station_a[0]]/vd
    delays[3] = max(delay_4)
    if waiting > max_waiting:
        delays[3] = 9999
    savings[delays[3]] = time_a_vh + time_b_vh - (dic[station_b[0]][station_a[0]] + 
                                                  dic[station_a[0]][station_a[1]] + 
                                                  dic[station_a[1]][station_b[1]])/vd
    if (min(delays)) <= max_delay:
        return True, savings[min(delays)], 0, 3

    # if none of the cases work, return infeasible
    return False, 0, 0, 999

In [340]:
def two_matching(demand,demand_time):
    RV_graph = nx.Graph()
    β = 0.1
    for i in range(0, len(demand)):
        for j in range(i+1, len(demand)):
            share, savings, incentives, case = check_feasibility(i, j, 600, 600, dic, demand_time, 
                                                                 pk_stop=60, dp_stop=60)
            if share:
                weight = β*savings - (1-β)*incentives
                RV_graph.add_edge(i, j, x=weight, saving = savings, incentive = incentives, case = case)
    return nx.max_weight_matching(RV_graph,weight='x'), RV_graph

In [341]:
pairs, RV_Graph = two_matching(demand_station, demand_time)

In [342]:
len(pairs)

82

In [162]:
def data_process(RV_Graph, pairs):
    total_incentives = 0
    total_savings = 0
    cases = np.zeros(4)
    for pair in pairs:
        total_incentives += RV_Graph[pair[0]][pair[1]]['incentive']
        total_savings += RV_Graph[pair[0]][pair[1]]['saving']
        cases[RV_Graph[pair[0]][pair[1]]['case']] += 1
    return total_incentives, total_savings, cases

In [343]:
t_i, t_s, cases = data_process(RV_Graph, pairs)

In [346]:
cases

array([  6.,  26.,  19.,  31.])

In [344]:
t_i/len(pairs)

1.405817028189179

In [345]:
t_s

27064.22055178562